# Búsqueda semántica con nubes de puntos
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \frac{1}{2|U|} \sum_{u \in U} \min_{x \in V} d(u, x) + \frac{1}{2|V|}\sum_{v \in V} \min_{x \in U} d(v, x) $$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{tp}$

$$ H^\textsf{pt}_d(U, V) = \max \left\{
        \max_{u \in U} \left\{\textsf{pt}(u, U) \min_{x \in V} d(u, x)\right\},
        \max_{v \in V} \left\{\textsf{pt}(v, V) \min_{x \in U} d(v, x)\right\}
     \right\} $$

# Notas adicionales
La versión de $H^\textsf{pt}_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")

using SimilaritySearch, TextSearch, Plots, Word2Vec, KNearestCenters, LinearAlgebra, HypertextLiteral, JSON, CodecZlib, Random
using Embeddings: load_embeddings, FastText_Text
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
include("knn.jl")

scores (generic function with 1 method)

In [3]:
function word2vec_embeddings(textconfig, corpus, embfile; dim=32, w2vtmp=tempfile(pwd()))
    isfile(embfile) && return
    tcorpus = tokenize_corpus(textconfig, corpus)
    voc = Vocabulary(textconfig, tcorpus)
    fvoc = filter_tokens(voc) do t
        5 <= t.ndocs <= 10000
    end
    valid = Set(fvoc.token)

    open(w2vtmp, "w") do f
        for tokens in tcorpus
            tokens = filter!(t -> t in valid, tokens)
            if length(tokens) > 7 # minimum number of tokens in a message
                println(f, join(tokens, ' '))
            end
        end
    end

    word2vec(w2vtmp, embfile, size=dim, iter=15, threads=Threads.nthreads()-1)
end


word2vec_embeddings (generic function with 1 method)

# Cargando datos y creando word embeddings a la medida

In [4]:
textconfig = TextConfig(; nlist=[1], qlist=[], slist=[], lc=true,
    group_usr=false, group_url=true, group_num=true, del_diac=true, del_punc=true)

display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_news()
embfile = "../data/news.w2v.vec"
isfile(embfile) && rm(embfile)
word2vec_embeddings(textconfig, D.corpus, embfile; dim=16, w2vtmp=joinpath(pwd(), "w2v.tmp"))
E = load_embeddings(FastText_Text, embfile)

@show unique(D.labels)

<h1>Cargando el corpus</h1>

Starting training using file /home/sadit/IR-2022/Unidades/w2v.tmp
Vocab size: 11080
Words in train file: 455119
unique(D.labels) = ["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau", "AristeguiOnline", "el_pais", "EFEnoticias", "PublimetroMX", "PublimetroChile", "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"]


25-element Vector{String}:
 "AdriDelgadoRuiz"
 "El_Universal_Mx"
 "CNNEE"
 "NTN24"
 "UniNoticias"
 "TelemundoNews"
 "SinEmbargoMX"
 "Reforma"
 "abc_es"
 "azucenau"
 "AristeguiOnline"
 "el_pais"
 "EFEnoticias"
 "PublimetroMX"
 "PublimetroChile"
 "Radio_Formula"
 "RicardoAlemanMx"
 "epigmenioibarra"
 "Milenio"
 "LaRazon_mx"
 "abrahamendieta"
 "PublimetroCol"
 "teleSURtv"
 "bbcmundo"
 "julioastillero"

In [5]:
T = let
    voc = Vocabulary(textconfig, D.corpus; thesaurus=E.vocab)
    model = VectorModel(BinaryGlobalWeighting(), TpWeighting(), voc)
    (; textconfig, model, voc)
end

(textconfig = TextConfig(true, false, true, true, true, false, false, true, Int8[], Int8[1], Skipgram[], IdentityTokenTransformation()), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=11080, train-n=30244, maxoccs=43239}, voc = Vocabulary(["</s>", "los", "del", "por", "un", "rt", "se", "con", "las", "para"  …  "marbelle", "cruces", "nolleva", "acaboelpaseviernes", "@docede", "@gregoriacarod", "salvadorena", "ignorancia", "nombrado", "mayorkas"], Int32[0, 8882, 7593, 6956, 6568, 6721, 6328, 5999, 5963, 5070  …  6, 6, 5, 5, 5, 5, 5, 5, 5, 5], Int32[0, 8882, 7593, 6956, 6568, 6721, 6328, 5999, 5963, 5070  …  6, 6, 5, 5, 5, 5, 5, 5, 5, 5], Dict{String, UInt32}("focos" => 0x00002284, "#videos" => 0x00001535, "libertad" => 0x00000213, "ayudas" => 0x00000f7a, "henry" => 0x000018e2, "islamico" => 0x000018d5, "directivos" => 0x00001c4d, "soporte" => 0x000013cb, "astronautas" => 0x00002532, "advirtieron" => 0x000024f4…), 30244))

In [6]:


cloud(text, T) = vectorize(T.model, T.textconfig, text; normalize=false)
cloud_corpus(corpus, T) = vectorize_corpus(T.model, T.textconfig, corpus; normalize=false)

struct AWMD{DistType} <: SemiMetric
    X::MatrixDatabase
    results::Vector{KnnResult}
    dist::DistType
end

@inline function nn_(X, res, dist, u, V)
    res = reuse!(res)

    for (vid, _) in V
        push!(res, vid, evaluate(dist, u, X[vid]))
    end
    
    minimum(res)
end

function onesidewmd(X, res, dist, U, V)
    d::Float32 = zero(Float32)
    @fastmath for (uid, w) in U
        if !haskey(V, uid) # evaluates to zero
            d = max(d, w * nn_(X, res, dist, X[uid], V))
        end
    end

    d
end

function SimilaritySearch.evaluate(wmd::AWMD, U::T, V::T) where T
    length(U) == 1 && haskey(U, 0) && return 1f3
    length(V) == 1 && haskey(V, 0) && return 1f3
    res = wmd.results[Threads.threadid()]

    max(onesidewmd(wmd.X, res, wmd.dist, U, V), onesidewmd(wmd.X, res, wmd.dist, V, U))
end

In [7]:
db = VectorDatabase(cloud_corpus(D.corpus, T))
#=for c in eachcol(E.embeddings)
    normalize!(c)
end=#
# dist = AWMD(MatrixDatabase(E.embeddings))
dist = AWMD(
    MatrixDatabase(E.embeddings),
    [KnnResult(1) for _ in 1:Threads.nthreads()],
    L2Distance()
)

AWMD{L2Distance}(MatrixDatabase{Matrix{Float32}}(Float32[0.025017 -2.122852 … -0.979149 -0.839573; 0.027621 0.398205 … 0.255373 0.433022; … ; -0.020723 -0.802804 … 0.393353 0.152764; -0.011764 -1.209199 … 0.187008 0.046162]), KnnResult[KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)  …  KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)], L2Distance())

In [8]:
#=recall = 0.8
index = SearchGraph(; dist, db, verbose=true)
#index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
index!(index)
optimize!(index, MinRecall(recall))
=#

In [9]:
index = ExhaustiveSearch(; db, dist);

In [16]:
let k = 1
    Qvectors = VectorDatabase(cloud_corpus(Q.corpus, T))
    t1 = @elapsed I, _ = searchbatch(index, Qvectors, k)
    s1 = scores(Q.labels, knn(I, D.labels))
    
    @htl """
    <div>search time: $t1, scores: $s1</div>
    """
end

<div>search time: 133.655987781, scores: <span class="Core-NamedTuple">(macrof1 = 0.27047964730664564, macrorecall = 0.281091825903312, accuracy = 0.3019934543290687)</span></div>

In [11]:
index = ParallelExhaustiveSearch(; db, dist);

In [12]:
#@time res, _ = search(ex, db[111], KnnResult(10))
#=@time res, _ = search(ex, cloud("covid en México", E, T), KnnResult(10))

for (id_, dist_) in res
    display(@htl """<div style="padding: 0.5em;">
        <span style="width: 25%; margin: 0.25em;"> $(id_ => round(dist_; digits=3))</span>
        <span style="background-color: rgb(120, 60, 60); margin: 0.2em;">$(D.text[id_])</span>
    </div>""")
end
=#

### Búsqueda y presentación de los resultados

In [13]:
function search_and_display(index, qtext, k, D, T)
    res = KnnResult(k)
    q = cloud(qtext, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados para "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [14]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, T)
search_and_display(index, "trafico de drogas", 7, D, T)
search_and_display(index, "calentamiento global", 7, D, T)
search_and_display(index, "covid corona virus", 7, D, T)


<h1>Ejemplos de búsqueda</h1>

  0.046767 seconds (120.95 k allocations: 2.769 MiB)


1,19737,0.261,azucenau,"La jefa de gobierno de la Ciudad de México, Claudia Sheinbaum, aseguró que ella y el presidente de la República, Andrés Manuel López Obrador, “somos parte del mismo movimiento”, y que jamás será líder de oposición, pues “somos parte de la Cuarta Transformación”. https://t.co/FYoh0GuROr"
2,27745,0.649,Reforma,"El Presidente Andrés Manuel López Obrador propuso al Gobierno de #NuevoLeon acordar con empresarios que produzcan menos, usen menos #agua, y que así vaya más líquido a la gente. https://t.co/9kW1cPZBmj"
3,19047,0.717,SinEmbargoMX,"El Secretario de Gobernación, Adán Augusto López, informó que el Gobierno federal y las aerolíneas llegaron a acuerdos para que los vuelos de carga doméstica y las operaciones tipo chárter migren “casi de inmediato” al AIFA https://t.co/LOJTQIPLxG https://t.co/41PU4vxjFX"
4,9748,0.742,SinEmbargoMX,"🚨 #URGENTE | El Gobierno de EU ofrece 5 millones de dólares por información que conduzca al arresto y condena de los mexicanos Ovidio Guzmán López, Iván Archivaldo Guzmán Salazar, Jesús Alfredo Guzmán Salazar y Joaquín Guzmán López, hijos de “El Chapo” https://t.co/S8eyJEDNTX https://t.co/YSXPkhUM7k"
5,22699,0.828,El_Universal_Mx,"RT @ElUniversalOax: #Entérate ➡️ A 11 días del impacto del huracán Agatha en #Oaxaca, el presidente Andrés Manuel López Obrador dijo que es…"
6,9958,0.828,El_Universal_Mx,#LaMañaneraDeAMLO El presidente López Obrador entrega a los doctores Jorge Salas Hernández y Manuel Martínez Lavín el reconocimiento al mérito médico 2021. 👉https://t.co/94pMxbgfFT https://t.co/kzILozOZmM
7,8555,0.828,EFEnoticias,"El presidente mexicano, Andrés Manuel López Obrador, anuncia que habrá regreso a clases presenciales ""llueva, truene o relampaguee"" pese a la tercera ola de covid-19 que aqueja al país y al recelo de algunos maestros. https://t.co/rA6FmtFASo"


  0.086905 seconds (120.93 k allocations: 2.768 MiB)


1,26942,0.289,TelemundoNews,"📌 #ENVIVO: Autoridades de Nueva York hablan de la extradición de Dairo Antonio Úsuga, uno de los capos más poderosos de Colombia. 👉🏻 El máximo jefe del Clan del Golfo, mejor conocido como Otoniel, debe responder por tráfico de drogas y armas. https://t.co/lDodg4jpcI"
2,19060,0.444,UniNoticias,"🇭🇳 El expresidente Juan Orlando Hernández, acusado de tráfico de drogas y armas, será extraditado a Estados Unidos el jueves 21 de abril a las 7.00 a. m., hora local. https://t.co/C4mHRVO1E1"
3,19869,0.445,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/jbAcSdIuPI"
4,10449,0.445,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/BNAdU3bp0n"
5,14765,0.445,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/cR2d88sSeF"
6,10549,0.445,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa88meses https://t.co/Yg7Rf8yDoS"
7,26176,0.445,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/cxi6tlCTMA"


  0.064159 seconds (120.93 k allocations: 2.768 MiB)


1,6080,0.345,SinEmbargoMX,"#Opinión | Ana Ortega de @THL_Latino ""El consumo de carne o lo que no se habla del calentamiento global"" Si no adoptamos una postura más firme ante el calentamiento global, el panorama de México está a punto de ponerse sombrío https://t.co/HL8v7EBp2x https://t.co/k8Az8hDr5y"
2,2189,0.406,CNNEE,"▶ Todavía podemos mitigar el daño al planeta por el calentamiento global si exigimos políticas públicas a gobiernos, dicen expertas. https://t.co/7eoc24sboY"
3,16524,0.703,CNNEE,"#podcast 🎧 La prioridad ahora para Washington no es Rusia, como muchos podrían pensar, sino un protagonista central, relativamente nuevo, de la estrategia global: China, calificado por EE.UU. como el mayor desafío al que se enfrenta ahora la humanidad. https://t.co/bBfj1oojY0"
4,4420,0.717,RicardoAlemanMx,RT @ContrapesoC: #Columna La geopolítica y el factor del cambio global Escribe @arturotaipan https://t.co/VepEkoQWUS
5,27948,0.744,bbcmundo,El calentamiento global avanza y los países parecen incapaces de cumplir con los objetivos de emisiones. https://t.co/xW9mKglT2u
6,12681,0.787,epigmenioibarra,"Histórico: OCDE amarra un acuerdo global para garantizar que las trasnacionales paguen una tasa impositiva mínima del 15%. De 140 países involucrados, 136 apoyaron el acuerdo; sólo se abstuvieron Kenia, Nigeria, Pakistán y Sri Lanka. vía @Forbes_Mexico https://t.co/xbBbdiF6t6 https://t.co/AejsxZYOTg"
7,19181,0.788,EFEnoticias,"La vicepresidenta estadounidense, Kamala Harris, instó este jueves a ""desafiar el statu quo"" y a actuar de forma conjunta a nivel global en la recuperación de la pandemia, de forma que puedan cerrarse las actuales desigualdades. https://t.co/OfEoj4s7fc"


  0.085491 seconds (120.93 k allocations: 2.768 MiB)


1,20260,0.967,CNNEE,"La humanidad ha logrado la inmunidad de rebaño contra algunos virus formidables, como la rubéola y el sarampión. Pensamos que podríamos llegar a ello con el covid-19. Probablemente nos equivocamos: https://t.co/ZA8T2sxwpY"
2,10574,0.992,El_Universal_Mx,"RT @De10mx: #HumorDe10 😂😷 La actriz Paty Navidad informó que dio positivo a Covid, pero insiste en que el virus no existe, por lo que los i…"
3,24688,1.022,abc_es,Un estudio de la Universidad de Córdoba relaciona las proteínas del virus del Covid con sus efectos más letales https://t.co/GWPqasZfHy
4,21905,1.026,bbcmundo,"¿Sabías que podrías ser una de las raras personas en el mundo que tienen una resistencia genética al SARS-Cov-2, el virus causante de la covid-19? https://t.co/SWj16nEITP"
5,25904,1.054,TelemundoNews,📌 #ENVIVO: Estudiantes en Chicago exigen mejores protocolos contra el COVID-19 para el regreso a clases presenciales. 👉🏻 Las escuelas públicas de la ciudad y el sindicato de maestros acordaron volver en medio de la rápida propagación del virus. https://t.co/Rpgmdp1gAw
6,11395,1.135,CNNEE,"China, el país en el que se detectó por primera vez el covid-19 hace casi dos años, sigue decidida a eliminar el virus dentro de sus fronteras, a pesar de haber vacunado a más del 75% de su población. https://t.co/uu2mbOFZHg"
7,29655,1.144,AristeguiOnline,Repuntan casos Covid en Hong Kong; ‘la ciudad no puede rendirse ante el virus’ https://t.co/udF8p2A9JK https://t.co/1cP410pc5g


In [15]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.174125 seconds (120.95 k allocations: 2.769 MiB)


1,24437,0.0,el_pais,"Este fin de semana se cambia la hora, pero eso no va a ayudar a la conciliación. La clave es que España no está en el huso horario que le corresponde, lo que se traduce en que los españoles llevan dos horas de retraso con el resto de Europa #hemeroteca https://t.co/gkVNV1jE2V"
2,6572,0.166,CNNEE,"Los damnificados por el huracán Ida, que regresan a sus casas dañadas, se enfrentan a un torrente de problemas, si es que tienen la suerte de tener una casa en pie. Mira cómo obtener ayuda y mantenerte a salvo en las difíciles semanas que se avecinan: https://t.co/en0lVMaEqd"
3,8569,0.167,EFEnoticias,"Las comunidades esperan para relajar el uso de mascarillas en interiores el criterio de los expertos de la Ponencia de Alertas y Planes de Preparación y Respuesta, que consideran que no debería retirarse hasta después de Semana Santa. https://t.co/hcymNQ7hJv https://t.co/IXO417klvL"
4,14582,0.168,el_pais,"La imagen se viralizó. Una embarazada con sangre en el rostro y gesto aturdido saliendo en pijama de un hospital recién bombardeado. Dio a luz al día siguiente. “Se encuentran bien, aunque hace mucho frío en Mariupol y los bombardeos no cesan” https://t.co/FQgBo5MkKN"
5,4361,0.178,el_pais,"🔊 El segundo día se lo dedicamos a Feijóo, candidato a las forzadas primarias del PP. Nos preguntamos quién es y cuál es su plan con dos de los periodistas que más le conocen. Porque, pese a sus años en la política, sigue siendo desconocido para muchos https://t.co/QRQ8SweTOQ https://t.co/KB0NUfatWV"
6,13975,0.182,el_pais,"Sánchez, sobre los indultos: “Lo que tenemos que hacer los políticos y los gobiernos es construir convivencia. En esa construcción hay momentos en los que lo útil es el castigo y otros en el que lo útil es el perdón. Ahora estamos en el momento del perdón” https://t.co/9QWlBqWaD5"
7,16344,0.182,CNNEE,"Desde el metaverso hasta los NFT, la nueva tecnología está cambiando sin cesar la forma en que vivimos nuestras vidas. Y uno de los escenarios más grandes donde se desarrolla es el deporte. Mira aquí algunos de los que ya existen https://t.co/jfe3LrEaXw"


  0.123769 seconds (120.97 k allocations: 2.769 MiB)


1,24734,0.0,EFEnoticias,"El Ministerio de Sanidad ha notificado 98 casos positivos de viruela del mono, que se distribuyen en siete comunidades autónomas, cuando se cumplen 10 días desde que el pasado 18 de mayo se confirmaran los primeros contagios. https://t.co/DR4EJs6CJF"
2,14053,0.213,SinEmbargoMX,"A mitad del sexenio de AMLO su talón de Aquiles sigue siendo el crecimiento económico, que lo ha quedado a deber, en gran medida, por el clima de recesión por el que atraviesa el país desde principios de 2019, mismo que se acentuó con la pandemia https://t.co/cjj7wceW3j https://t.co/Jpm3WZdTB2"
3,27208,0.217,el_pais,"La pandemia ha servido a youtubers educativos como @QuantumFracture, @davidcpvm o @math2me para validar lo que llevan años haciendo y ha abierto la puerta a que otros profesores se animen a compartir en un canal sus conocimientos https://t.co/f5WRsZMRg1"
4,16662,0.218,AdriDelgadoRuiz,"“La realidad es que de cualquier manera renunció y Coumo se encontraba en medio de un escándalo general, había muchos cuestionamientos respecto al manejo de la pandemia, particularmente en lo referente a los asilos para los ancianos del estado”"
5,5565,0.224,EFEnoticias,"Un total de 2.520 nuevos contagios y siete fallecidos más por COVID se han registrado en las últimas 24 horas en Cataluña, donde además ha aumentado el riesgo de rebrote y el número de pacientes ingresados en las unidades de cuidados intensivos (UCI). https://t.co/jghE1YS7qc"
6,5964,0.227,TelemundoNews,"El pequeño Wyatt Gibson, de 5 años, murió de COVID-19 en Georgia mientras los casos aumentan en el país. ""Había tenido algún resfriado pero nada más. (El virus) se apoderó de él y se lo llevó. Es una locura"", aseguró su madrina. https://t.co/4Mwrl0Asxh"
7,11056,0.23,teleSURtv,"#ENVIDEO📹| La Organización Mundial de la Salud advierte sobre el aumento preocupante de casos de #HepatitisAguda en niños Casi 200 casos han sido detectados en más de 10 países y aún se desconoce la verdadera causa de la enfermedad Conoce más, en este video→ https://t.co/koQj6ocDPX"


  0.114441 seconds (120.95 k allocations: 2.768 MiB)


1,20829,0.0,EFEnoticias,"El líder del PSC en el Parlament, Salvador Illa, ha reclamado al presidente de la Generalitat, Pere Aragonès, que acuda a la conferencia de presidentes autonómicos prevista para este mes de julio en Salamanca. https://t.co/yUnsJNyQLb"
2,9725,0.226,EFEnoticias,"El presidente del Gobierno, Pedro Sánchez, ha anunciado este miércoles, en un acto de apertura del curso político, que el Ejecutivo tiene la voluntad de abordar la ""subida inmediata"" del salario mínimo interprofesional ""en lo que queda de año"". https://t.co/W9PR9uHzFL https://t.co/7l4D6NZw5f"
3,8521,0.233,el_pais,"El Gobierno italiano ha decretado el uso de mascarillas al aire libre, será obligatorio utilizar la FFP2 en los transportes y otros lugares concurridos y también cerrarán las discotecas y salas de baile hasta el 31 de enero https://t.co/DJy9uVXXxC"
4,755,0.235,EFEnoticias,"El Senado de Haití instó al primer ministro, Ariel Henry, a entregar el poder a la Cámara alta el próximo lunes 7 de febrero, fecha en la que habría finalizado el mandato del presidente Jovenel Moise, asesinado en julio de 2021. https://t.co/pr4y8Anet3"
5,1333,0.235,Reforma,"Estos son los temas que trató el Presidente @lopezobrador_ en la mañanera del 13 de diciembre. -'Empresarios han ofrecido disculpas por reforma eléctrica' -'Despropósito, comparar conflicto de CIDE con 68' -Anuncia en víspera de elecciones obras en Cancún y Tulum Revisa el hilo https://t.co/b8pzyp449A"
6,14053,0.235,SinEmbargoMX,"A mitad del sexenio de AMLO su talón de Aquiles sigue siendo el crecimiento económico, que lo ha quedado a deber, en gran medida, por el clima de recesión por el que atraviesa el país desde principios de 2019, mismo que se acentuó con la pandemia https://t.co/cjj7wceW3j https://t.co/Jpm3WZdTB2"
7,2994,0.238,el_pais,"🔴 ÚLTIMA HORA | El congreso del PP proclama a Feijóo nuevo presidente del partido con un 98,35% de votos a favor. Este ha sido el momento en el que se ha anunciado el resultado https://t.co/EnOIovl4ls https://t.co/2Ck9ECKr8g"


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^\textsf{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Defina $H^\textsf{TF}_d$, donde $TF$ es la frecuencia de término normalizada.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
